<a href="https://colab.research.google.com/github/sholtodouglas/learning_from_play/blob/master/experimental/Generate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -q
!pip install pathy -q
!pip install comet_ml -q

     |████████████████████████████████| 2.0MB 11.9MB/s 
     |████████████████████████████████| 133kB 49.0MB/s 
     |████████████████████████████████| 163kB 44.8MB/s 
     |████████████████████████████████| 102kB 6.3MB/s 
     |████████████████████████████████| 71kB 4.4MB/s 
     |████████████████████████████████| 122kB 12.6MB/s 
     |████████████████████████████████| 256kB 8.8MB/s 
     |████████████████████████████████| 512kB 25.9MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 204kB 31.8MB/s 


In [12]:
import argparse

parser = argparse.ArgumentParser(description='LFP training arguments')
parser.add_argument('run_name')
parser.add_argument('--train_datasets', nargs='+', help='Training dataset names')
parser.add_argument('--test_datasets', nargs='+', help='Testing dataset names')
parser.add_argument('-c', '--colab', default=False, action='store_true', help='Enable if using colab environment')
parser.add_argument('-s', '--data_source', default='DRIVE', help='Source of training data')
parser.add_argument('-tfr', '--from_tfrecords', default=False, action='store_true', help='Enable if using tfrecords format')
parser.add_argument('-d', '--device', default='TPU', help='Hardware device to train on')
parser.add_argument('-b', '--batch_size', default=512, type=int)
parser.add_argument('-wmax', '--window_size_max', default=40, type=int)
parser.add_argument('-wmin', '--window_size_min', default=20, type=int)
parser.add_argument('-la', '--actor_layer_size', default=2048, type=int, help='Layer size of actor, increases size of neural net')
parser.add_argument('-le', '--encoder_layer_size', default=512, type=int, help='Layer size of encoder, increases size of neural net')
parser.add_argument('-lp', '--planner_layer_size', default=512, type=int, help='Layer size of planner, increases size of neural net')
parser.add_argument('-embd', '--img_embedding_size', default=64, type=int, help='Embedding size of features,goal space')
parser.add_argument('-z', '--latent_dim', default=256, type=int, help='Size of the VAE latent space')
parser.add_argument('-g', '--gcbc', default=False, action='store_true', help='Enables GCBC, a simpler model with no encoder/planner')
parser.add_argument('-n', '--num_distribs', default=None, type=int, help='Number of distributions to use in logistic mixture model')
parser.add_argument('-q', '--qbits', default=None, type=int, help='Number of quantisation bits to discrete distributions into. Total quantisations = 2**qbits')
parser.add_argument('-lr', '--learning_rate', type=float, default=3e-4)
parser.add_argument('-t', '--train_steps', type=int, default=200000)
parser.add_argument('-r', '--resume', default=False, action='store_true')
parser.add_argument('-B', '--beta', type=float, default=0.00003)
parser.add_argument('-i', '--images', default=False, action='store_true')
parser.add_argument('--bucket_name', help='GCS bucket name to stream data from')
parser.add_argument('--tpu_name', help='GCP TPU name') # Only used in the script on GCP

### Sample local config
args = parser.parse_args('''
dummy_run 
--train_dataset UR5
--test_dataset UR5
-tfr
'''.split())

## Sample colab config
args = parser.parse_args('''
refactor_test
--train_dataset UR5 UR5_slow_gripper UR5_high_transition
--test_dataset UR5_slow_gripper_test
-c
-tfr
-s GCS
-d TPU
-b 512
-la 2048
-le 512
-lp 512
-z 256
-lr 3e-4
--bucket_name iowa_bucket_lfp
'''.split())

# ## Sample colab config
# args = parser.parse_args('''
# QuantB0_01
# --train_dataset UR5 UR5_slow_gripper UR5_high_transition
# --test_dataset UR5_slow_gripper_test
# -c
# -s DRIVE
# -d TPU
# -b 512
# -la 2048
# -le 512
# -lp 512
# -z 256
# -lr 3e-4
# -B 0.01
# -n 5
# -q 8
# '''.split())

print(args)

Namespace(actor_layer_size=2048, batch_size=512, beta=3e-05, bucket_name='iowa_bucket_lfp', colab=True, data_source='GCS', device='TPU', encoder_layer_size=512, from_tfrecords=True, gcbc=False, images=False, img_embedding_size=64, latent_dim=256, learning_rate=0.0003, num_distribs=None, planner_layer_size=512, qbits=None, resume=False, run_name='refactor_test', test_datasets=['UR5_slow_gripper_test'], tpu_name=None, train_datasets=['UR5', 'UR5_slow_gripper', 'UR5_high_transition'], train_steps=200000, window_size_max=40, window_size_min=20)


In [ ]:
from pathlib import Path
from pathy import Pathy
import os
import requests
import json
import pprint
import logging
import numpy as np
import tensorflow as tf
import time



pp = pprint.PrettyPrinter(indent=4)
# In[4]:


#@title Workpace Setup (Local vs Colab)

# Set up working directory and libraries
if args.colab:
    from google.colab import drive, auth
    print('Using colab setup')
    WORKING_PATH = Path('/content/learning_from_play')
    # Clone repo
    try:
        get_ipython().system("git clone 'https://github.com/sholtodouglas/learning_from_play' {WORKING_PATH}")
    except: 
        pass
    # Mount drive
    drive.mount('/content/drive')
else:
    print('Using local setup')
    WORKING_PATH = Path.cwd()
    print(f'Working path: {WORKING_PATH}')

# Change working directory to learning_from_play
os.chdir(WORKING_PATH)
import lfp

# Set up storage directory and datasets
if args.data_source == 'DRIVE':
    assert args.colab, "Must be using Colab"
    print('Reading data from Google Drive')
    STORAGE_PATH = Path('/content/drive/My Drive/Robotic Learning')
elif args.data_source == 'GCS':
    if args.colab:
      auth.authenticate_user()
    print('Reading data from Google Cloud Storage')
    r = requests.get('https://ipinfo.io')
    region = r.json()['region']
    project_id = 'learning-from-play-303306'
    logging.warning(f'You are accessing GCS data from {region}, make sure this is the same as your bucket {args.bucket_name}')
    STORAGE_PATH = Pathy(f'gs://{args.bucket_name}')
else:
    print('Reading data from local filesystem')
    STORAGE_PATH = WORKING_PATH

print(f'Storage path: {STORAGE_PATH}')
TRAIN_DATA_PATHS = [STORAGE_PATH/'data'/x for x in args.train_datasets]
TEST_DATA_PATHS = [STORAGE_PATH/'data'/x for x in args.test_datasets]

Using colab setup
Cloning into '/content/learning_from_play'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 2217 (delta 77), reused 82 (delta 41), pack-reused 2088
Receiving objects: 100% (2217/2217), 82.48 MiB | 7.54 MiB/s, done.
Resolving deltas: 100% (1314/1314), done.
Mounted at /content/drive
No pybullet installation found - which is fine if training
Reading data from Google Drive
Storage path: /content/drive/My Drive/Robotic Learning


In [6]:

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


In [13]:
dl = lfp.data.PlayDataloader(include_imgs = args.images, batch_size=1,  window_size=args.window_size_max, min_window_size=args.window_size_min)

In [14]:
# Train data
train_data = dl.extract(TRAIN_DATA_PATHS, from_tfrecords=args.from_tfrecords)
train_dataset = dl.load(train_data)

{   'acts': TensorSpec(shape=(1, 40, 7), dtype=tf.float32, name=None),
    'dataset_path': TensorSpec(shape=(1, None), dtype=tf.int32, name=None),
    'goals': TensorSpec(shape=(1, 40, 11), dtype=tf.float32, name=None),
    'masks': TensorSpec(shape=(1, 40), dtype=tf.float32, name=None),
    'obs': TensorSpec(shape=(1, 40, 18), dtype=tf.float32, name=None),
    'seq_lens': TensorSpec(shape=(1,), dtype=tf.float32, name=None),
    'tstep_idxs': TensorSpec(shape=(1, None), dtype=tf.int32, name=None)}


In [ ]:
def serialise(data):
    
    obs, acts, achieved_goals, joint_poses, target_poses, acts_quat, acts_rpy_rel, velocities, obs_quat, gripper_proprioception, sequence_index, sequence_id, img = data['obs'], data['acts'], data['achieved_goals'], data['joint_poses'], data['target_poses'], data['acts_quat'], data['acts_rpy_rel'], data['velocities'], data['obs_quat'], data['gripper_proprioception'], data['sequence_index'], data['sequence_id'], data['img']
    
    #image_feature = Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(image).numpy(),]))
    
    obs = Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(obs).numpy(),]))
    acts = Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(acts).numpy(),]))
    achieved_goals = Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(achieved_goals).numpy(),])) 
    joint_poses = Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(joint_poses).numpy(),])) 
    target_poses = Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(target_poses).numpy(),]))
    acts_quat = Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(acts_quat).numpy(),]))
    acts_rpy_rel = Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(acts_rpy_rel).numpy(),])) 
    velocities = Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(velocities).numpy(),])) 
    obs_quat = Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(obs_quat).numpy(),]))
    gripper_proprioception = Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(gripper_proprioception).numpy(),])) 
    sequence_index =  Feature(int64_list=Int64List(value=[sequence_index,]))
    sequence_id = Feature(int64_list=Int64List(value=[sequence_id,]))
    img = Feature(bytes_list=BytesList(value=[img.numpy(),]))
    # img is already serialised because we never decode it!
    
    features = Features(feature={
                'obs': obs,
                'acts': acts,
                'achieved_goals': achieved_goals,
                'joint_poses': joint_poses,
                'target_poses': target_poses,
                'acts_quat': acts_quat,
                'acts_rpy_rel': acts_rpy_rel,
                'velocities': velocities,
                'obs_quat': obs_quat,
                'gripper_proprioception': gripper_proprioception,
                'sequence_index': sequence_index,
                'sequence_id': sequence_id,
                'img': img,
                })
    
    example = Example(features=features)
    
    return example.SerializeToString()






with tf.io.TFRecordWriter(str(path/'tf_records/')+f"/{demo}.tfrecords") as file_writer:
            for data in it:
                byte_stream = serialise(data)
                file_writer.write(byte_stream)